In [1]:
# imports
from tqdm.notebook import tqdm

import os
data_dir = './data'
if not os.path.exists(data_dir): os.makedirs(data_dir)

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
start = {'AMZN': '1/4/21', 'NFLX': '2/1/21', 'TSLA': '1/4/21'}

for ticker in tqdm(['AMZN', 'NFLX', 'TSLA'], leave=True):
    # read in trade (not bid or ask) data
    df = pd.read_csv(f'./data/{ticker}.csv', header=3, usecols=range(8))
    # drop NAs caused by bid and/or ask data
    df.dropna(subset='Dates', inplace=True)
    # fix first row with invalid date
    df.loc[0, 'Dates'] = start[ ticker ] + ' 9:30'
    # Dates col to datetime
    df['Dates'] = pd.to_datetime(df.Dates)
    df.set_index('Dates', inplace=True)

    # bounds
    x, ub = df.index.min(), df.index.max()
    y = x + pd.DateOffset(hours=2, minutes=30)
    day = pd.DateOffset(days=1)

    returns = dict()
    i = 0
    # iterating over days
    while x < ub and y < ub:
        try:
            open = df.loc[x, 'Open']
            close = df.loc[y, 'Close']
            returns[y] = close / open - 1
        except: i += 1

        x, y = x + day, y + day

    if i > 0: print(f'Failed to parse {i} returns for {ticker}.')

    # save returns per ticker as defined in HW2
    returns = pd.DataFrame.from_dict(returns, orient='index')
    returns.columns = ['Returns']
    returns.to_csv(f'./data/{ticker}_processed.csv')

  0%|          | 0/3 [00:00<?, ?it/s]

Failed to parse 61 returns for AMZN.
Failed to parse 58 returns for NFLX.
Failed to parse 59 returns for TSLA.


In [20]:
TSLA = pd.read_csv("./data/TSLA.csv")

In [21]:
TSLA

,BarTp,Trade,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,BarTp.1,...,Unnamed: 16,Unnamed: 17,BarTp.2,Ask,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TSLA US Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TSLA US Equity,...,NaN,NaN,TSLA US Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dates,Open,Close,High,Low,Value,Volume,Number Ticks,NaN,Dates,...,Number Ticks,NaN,Dates,Open,Close,High,Low,Value,Volume,Number Ticks
3,1/4/21 9:30,719.46,717.68,721.49,717.19,529887552,736210,1186,NaN,1/4/21 3:59,...,12,NaN,1/4/21 3:59,750,710,750,709.29,18044.529,25,15
4,1/4/21 9:31,717.87,719.38,721.57,717.19,136473008,189614,1077,NaN,1/4/21 4:00,...,71,NaN,1/4/21 4:00,710,717,717,709.95,81167.242,114,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7/13/21 15:57,...,540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7/13/21 15:58,...,531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7/13/21 15:59,...,819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7/13/21 16:00,...,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
TSLA_ret = pd.read_csv("./data/TSLA_processed.csv")

r_bar = TSLA_ret.Returns.mean()

r_bar


-0.00036371742488483326

In [19]:
TSLA_ret

,Unnamed: 0,Returns
0,2021-01-04 12:00:00,0.015539
1,2021-01-05 12:00:00,0.018572
2,2021-01-06 12:00:00,0.018009
3,2021-01-07 12:00:00,0.028065
4,2021-01-08 12:00:00,0.020783
...,...,...
127,2021-07-07 12:00:00,-0.026224
128,2021-07-08 12:00:00,0.027068
129,2021-07-09 12:00:00,-0.001508
130,2021-07-12 12:00:00,0.026679
